<a href="https://colab.research.google.com/github/rlancaster243/Machine-Learning-FreeCodeCamp/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Preprocess Data
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# Identify categorical and numerical columns
categorical_columns = ['sex', 'smoker', 'region']
numerical_columns = ['age', 'bmi', 'children']

# Ensure dataset columns are correctly named
print("Dataset Columns:", dataset.columns)
dataset.columns = dataset.columns.str.strip().str.lower()

# Check if categorical columns exist before encoding
if all(col in dataset.columns for col in categorical_columns):
    dataset = pd.get_dummies(dataset, columns=categorical_columns, drop_first=True)
else:
    print("Categorical columns have already been encoded or are missing.")

# Split features and target
train_dataset = dataset.drop(columns=['expenses'])
train_labels = dataset['expenses']

# Split into training and testing sets (80-20 split)
train_dataset, test_dataset, train_labels, test_labels = train_test_split(train_dataset, train_labels, test_size=0.2, random_state=42)

# Scale numerical features
scaler = StandardScaler()
train_dataset[numerical_columns] = scaler.fit_transform(train_dataset[numerical_columns])
test_dataset[numerical_columns] = scaler.transform(test_dataset[numerical_columns])

# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(train_dataset.shape[1],)),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae', 'mse'])

# Train the model
history = model.fit(train_dataset, train_labels, validation_data=(test_dataset, test_labels), epochs=100, batch_size=32)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
